In [4]:
! pip install lxml
! pip install html5lib
! pip install BeautifulSoup4
! pip install tabulate

     |████████████████████████████████| 5.8MB 25.8MB/s eta 0:00:01
     |████████████████████████████████| 102kB 18.4MB/s ta 0:00:01
     |████████████████████████████████| 51kB 12.1MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/e1/41/5e/e201f95d90fc84f93aa629b6638adacda680fe63aac47174ab
Successfully built tabulate


In [5]:
from bs4 import BeautifulSoup
from tabulate import tabulate
import pandas as pd
import requests

#### Read data

In [43]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
df = pd.DataFrame(df[0])
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Remove rows with Borough = "Not assigned"

In [53]:
df = df[df['Borough'] != 'Not assigned' ]
df.head()
df.shape

(211, 3)

#### Aggregate the data by Postcode & Borough

In [62]:
g_df = df.groupby(['Postcode', 'Borough'], as_index=False).agg(lambda col: ', '.join(col))
g_df


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


#### Assign Borough to Neighbourhood if not assigned

In [99]:
df_final = g_df
num = df_final.shape[0]
Nghb = [None] * num

for i in range(0, num):
    if df_final.loc[i, 'Neighbourhood']== 'Not assigned':
        Nghb[i]=df_final.loc[i, 'Borough']
    else:
        Nghb[i]=df_final.loc[i, 'Neighbourhood']
df_final['Neighbourhood'] = Nghb


#### Check the final table

In [102]:
df_final.shape

(103, 3)

In [101]:
print(df_final.to_string())

    Postcode           Borough                                      Neighbourhood
0        M1B       Scarborough                                     Rouge, Malvern
1        M1C       Scarborough             Highland Creek, Rouge Hill, Port Union
2        M1E       Scarborough                  Guildwood, Morningside, West Hill
3        M1G       Scarborough                                             Woburn
4        M1H       Scarborough                                          Cedarbrae
5        M1J       Scarborough                                Scarborough Village
6        M1K       Scarborough        East Birchmount Park, Ionview, Kennedy Park
7        M1L       Scarborough                    Clairlea, Golden Mile, Oakridge
8        M1M       Scarborough    Cliffcrest, Cliffside, Scarborough Village West
9        M1N       Scarborough                        Birch Cliff, Cliffside West
10       M1P       Scarborough  Dorset Park, Scarborough Town Centre, Wexford ...
11       M1R    